In [ ]:
import pandas as pd
from wormstools.worms_core import Worms

In [ ]:
# See database_helpers.py
run database_helpers.py

In [ ]:
# First, get all of the species names from the raw data
species = select("""
SELECT DISTINCT "scientific_name" from "Raw_Fact";
""")
species = [row[0] for row in species]

In [ ]:
# This is the list of ranks we will access, not every lookup will have every level
taxonomic_ranks = ["Kingdom", "Phylum", "Class", "Order", "Family", "Genus"]
# Create a container for the data from lookups
taxonomicData = pd.DataFrame({}, columns=taxonomic_ranks + ["Species"])

# then iterate through lookup calls and build the taxa table data
# There are >2000 species, this takes some time!
for speciesName in species:
    # get an specific taxonomic rank
    lookup = Worms(taxon = speciesName)
    lookup.get_taxonomic_ranges()
    try:
        speciesData = {rank:(lookup.get_rank(rank) if rank in lookup.taxonomic_ranges else None) for rank in taxonomic_ranks}
        speciesData['Species'] = speciesName
        taxonomicData = taxonomicData.append(pd.DataFrame([speciesData]))
    except:
        print("Species not found `{}`".format(speciesName))

In [ ]:
# Write this data to file, Other steps in the ETL pipeline
# will insert this data into the table
taxonomicData.to_csv("../Data/taxonomy.csv")